In [1]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.set_printoptions(linewidth=10000)
tokenizer_name = 'bert-base-multilingual-cased'
# tokenizer_name = 'microsoft/mdeberta-v3-base'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
# model_name = 'pgajo/mbert-xl-wa-tasteset-recipe-aligner'
# model_name = 'pgajo/mdeberta-v3-base-recipe-aligner'
# model_name = 'pgajo/mdeberta-v3-base-xl-wa-tasteset-recipe-aligner'
# model_name = 'pgajo/mdeberta-v3-base-xl-wa_TASTEset_20240101-140212'
# model_name = 'pgajo/bert-base-multilingual-cased-recipe-aligner-en-it-3-epochs'
model_name = 'pgajo/bert-base-multilingual-cased-xl-wa-it-5-epochs'
model = AutoModelForQuestionAnswering.from_pretrained(model_name).to(device)

/home/pgajo/working/food/food-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 841/841 [00:00<00:00, 621kB/s]
model.safetensors: 100%|██████████| 709M/709M [00:14<00:00, 50.6MB/s] 


Create bilingual version of the json dataset

In [2]:
import json
# path = '/home/pgajo/working/food/data/TASTEset/data/TASTEset_sep_format.json'
# with open(path, encoding='utf8') as f:
#     data = json.load(f)
# print(data)

# with open('/home/pgajo/working/food/data/TASTEset/data/TASTEset_sep_format_raw.nofractions.it', 'r', encoding='utf8') as f:
#     italian_recipes = f.readlines()
# bilingual_data = []
# for i, entry in enumerate(data['annotations']):
#     new_entry = {}
#     new_entry['text_en'] = entry['text_en']
#     new_entry['entities_en'] = entry['entities_en']
#     new_entry['text_it'] = italian_recipes[i].strip()
#     new_entry['entities_it'] = []
#     bilingual_data.append(new_entry)
# bilingual_dataset = {'classes': data['classes'], 'annotations': bilingual_data}
# print(bilingual_dataset)

# save bilingual dataset to json
# bilingual_path = path.replace('.json', '_en-it_unaligned.json')
# with open(bilingual_path, 'w', encoding='utf8') as f:
    # json.dump(bilingual_dataset, f, ensure_ascii=False)

Inference

In [3]:
import json
from tqdm.auto import tqdm
bilingual_path = '/home/pgajo/working/food/data/TASTEset/data/formatted data/TASTEset_sep_format_en-it_unaligned.json'
with open(bilingual_path, encoding='utf8') as f:
    data = json.load(f)

print(data['annotations'][:3])

recipe_list = data['annotations'][:3]
len(recipe_list)

[{'text_en': '5 ounces rum;4 ounces triple sec;3 ounces Tia Maria;20 ounces orange juice', 'entities_en': [[0, 1, 'QUANTITY'], [2, 8, 'UNIT'], [9, 12, 'FOOD'], [13, 14, 'QUANTITY'], [15, 21, 'UNIT'], [22, 32, 'FOOD'], [33, 34, 'QUANTITY'], [35, 41, 'UNIT'], [42, 51, 'FOOD'], [52, 54, 'QUANTITY'], [55, 61, 'UNIT'], [62, 74, 'FOOD']], 'text_it': "5 once di rum;4 once di triple sec;3 once di Tia Maria;20 once di succo d'arancia", 'entities_it': []}, {'text_en': '2 tubes cinnamon roll, refrigerated, with icing;4 tablespoons butter, melted;6 eggs;1/2 cup milk;2 teaspoons cinnamon;2 teaspoons vanilla;1 cup maple syrup', 'entities_en': [[0, 1, 'QUANTITY'], [2, 7, 'UNIT'], [8, 21, 'FOOD'], [23, 35, 'PROCESS'], [37, 41, 'FOOD'], [42, 47, 'FOOD'], [48, 49, 'QUANTITY'], [50, 61, 'UNIT'], [62, 68, 'FOOD'], [70, 76, 'PROCESS'], [77, 78, 'QUANTITY'], [79, 83, 'FOOD'], [84, 87, 'QUANTITY'], [88, 91, 'UNIT'], [92, 96, 'FOOD'], [97, 98, 'QUANTITY'], [99, 108, 'UNIT'], [109, 117, 'FOOD'], [118, 119, 'QU

3

In [4]:
for idx, recipe in tqdm(enumerate(recipe_list), total=len(recipe_list)):
    for i, entity in enumerate(recipe['entities_en']):
        input = tokenizer(
            recipe['text_en'][:entity[0]] + '• ' + recipe['text_en'][entity[0]:entity[1]] + ' •' + recipe['text_en'][entity[1]:],
            recipe['text_it'],
            return_tensors='pt',
            ).to('cuda')
        input_ids = input['input_ids'].squeeze()
        with torch.inference_mode():
            outputs = model(**input)
        start_scores = outputs.start_logits
        end_scores = outputs.end_logits
        start_index_token = int(torch.argmax(start_scores))
        # print('start_index_token', start_index_token)
        end_index_token = int(torch.argmax(end_scores))
        # print('end_index_token', end_index_token)
        # print('len(input_ids)', len(input_ids))
        if start_index_token >= len(input_ids) - 1 or end_index_token >= len(input_ids) - 1:
            continue
        # print('encoding:', input_ids)
        decoded_input = tokenizer.decode(input_ids)
        # print('decoded:', decoded_input)
        # for j, id in enumerate(input_ids):
        #     print(j, int(id), tokenizer.decode([id]), end='\t\t')
        # print()
        # print('prediction_tokens:', input['input_ids'].squeeze()[start_index_token:end_index_token])
        # print('prediction:', tokenizer.decode(input['input_ids'].squeeze()[start_index_token:end_index_token]))
        # print('gold:', [recipe['text_en'][entity[0]:entity[1]]])
        char_span_start = input.token_to_chars(start_index_token)
        # print('char_span_start', char_span_start)
        char_span_end = input.token_to_chars(end_index_token-1)
        char_span_prediction = recipe['text_it'][char_span_start[0]:char_span_end[1]]
        char_span_prediction_splitjoined = ''.join(char_span_prediction.split()).replace('#', '')
        # print('char_span_prediction', )
        # print('char_span_end', char_span_end)
        char_span = (char_span_start[0], char_span_end[1])
        # print('char_span', char_span)
        # print('char_span[0]', char_span[0])
        # print('char_span[1]', char_span[1])
        if not char_span[0] > char_span[1]:
            recipe['entities_it'].append([char_span[0], char_span[1], recipe['entities_en'][i][2]])
            # print(recipe['entities_it'])
        # else:
            # print('skipping')
        
        token_based_prediction = tokenizer.decode(input['input_ids'].squeeze()[start_index_token:end_index_token])
        token_based_prediction_splitjoined = ''.join(token_based_prediction.split()).replace('#', '')

        print('full context:', recipe['text_it'])
        print('encoding:', input_ids)
        print('decoded:', decoded_input)
        print('prediction_tokens:', input['input_ids'].squeeze()[start_index_token:end_index_token])
        print('char_span_prediction:', [char_span_prediction])
        print('char_span_prediction_splitjoined:', [char_span_prediction_splitjoined]) 
        print('token_based_prediction:', [token_based_prediction])
        print('token_based_prediction_splitjoined:', [token_based_prediction_splitjoined])
        print('gold:', [recipe['text_en'][entity[0]:entity[1]]])

        if char_span_prediction_splitjoined != token_based_prediction_splitjoined:
            print('ERROR: CHAR SPAN PREDICTION DOES NOT MATCH TOKEN-BASED PREDICTION')
            # print(f'################# SKIPPING #################\nerrors: {error_count+1}\nerror rate = {error_count/sample_count}')
            error = True
            print(f'recipe no.: {idx}, entity no.: {i}')
            print(recipe['text_en'][entity[0]:entity[1]])
            print(recipe['text_it'])
            print('start_index_token:', start_index_token)
            print('end_index_token:', end_index_token)
            print('len(input_ids):', len(input_ids))
            print('encoding:', input_ids)
            print('decoded:', decoded_input)
            print('prediction_tokens:', input['input_ids'].squeeze()[start_index_token:end_index_token])
            print('token_based_prediction:', [token_based_prediction])
            print('token_based_prediction_splitjoined:', [token_based_prediction_splitjoined])
            print('gold:', [recipe['text_en'][entity[0]:entity[1]]])
            print('char_span_start', char_span_start)
            print('char_span_end', char_span_end)
            print('char_span_prediction:', [char_span_prediction])
            print('char_span_prediction_splitjoined:', [char_span_prediction_splitjoined])
            print('full context_en:', recipe['text_en'])
            print('full context_it:', recipe['text_it'])
            break
print(data)

  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [00:00<00:00,  2.99it/s]

full context: 5 once di rum;4 once di triple sec;3 once di Tia Maria;20 once di succo d'arancia
encoding: tensor([  101,  1729,   126,  1729, 10431, 28524, 52522,   132,   125, 10431, 28524, 40159, 37913,   132,   124, 10431, 28524, 29033, 10113, 11066,   132, 10197, 10431, 28524, 41435, 23005, 11918,   102,   126, 14907, 10120, 52522,   132,   125, 14907, 10120, 40159, 37913,   132,   124, 14907, 10120, 29033, 10113, 11066,   132, 10197, 14907, 10120, 10198, 20493,   172,   112, 13785, 13212,   102], device='cuda:0')
decoded: [CLS] • 5 • ounces rum ; 4 ounces triple sec ; 3 ounces Tia Maria ; 20 ounces orange juice [SEP] 5 once di rum ; 4 once di triple sec ; 3 once di Tia Maria ; 20 once di succo d'arancia [SEP]
prediction_tokens: tensor([126], device='cuda:0')
char_span_prediction: ['5']
char_span_prediction_splitjoined: ['5']
token_based_prediction: ['5']
token_based_prediction_splitjoined: ['5']
gold: ['5']
full context: 5 once di rum;4 once di triple sec;3 once di Tia Maria;20 on

 67%|██████▋   | 2/3 [00:00<00:00,  2.97it/s]

full context: 2 tubi di cinnamon roll, refrigerati, con glassa;4 cucchiai di burro, fuso;6 uova;1/2 tazza di latte;2 cucchiaini di cannella;2 cucchiaini di vaniglia;1 tazza di sciroppo d'acero
encoding: tensor([  101,   123, 79013, 11322, 16441, 16278, 25520,   117, 48056, 45854, 32199,   117, 10169, 42032, 10376,   132,   125, 61939, 74378, 10107, 10473, 10877,   117, 10911, 17098, 10162,   132,   127, 58511,   132,  1729,   122,   120,   123,  1729, 41506, 64301,   132,   123, 57675, 54609, 89305, 11322, 16441, 16278,   132,   123, 57675, 54609, 89305, 10145, 14250,   132,   122, 41506, 14876, 10284, 12261, 36660,   102,   123, 13055, 11645, 10120, 11322, 16441, 16278, 25520,   117, 48056, 45854, 22425,   117, 10173, 32362, 10113,   132,   125, 10854, 63641, 10116, 10120, 11499, 38692,   117, 11005, 11669,   132,   127,   189, 12804,   132,   122,   120,   123, 11057, 15003, 10120, 12764, 10216,   132,   123, 10854, 63641, 11778, 10120, 10944, 49109,   132,   123, 10854, 63641, 11778

100%|██████████| 3/3 [00:00<00:00,  3.15it/s]

full context: 4 noci di cocco mature;1 tazza di latte evaporato;1 tazza di gin;3 cucchiai di zucchero (facoltativo);1 cucchiaino di cannella macinata;1/2 cucchiaino di noce moscata fresca grattugiata
encoding: tensor([   101,    125,  29956,  11355,  11170,  23486,  33876,    132,    122,  41506, 103730,  25130,  11912,  64301,    132,    122,  41506,  38356,  10115,    132,   1729,    124,   1729,  61939,  74378,  10107,  60390,    113,  96645,    114,    132,    122,  57675,  54609,  15938,  16912,  11322,  16441,  16278,    132,    122,    120,    123,  57675,  54609,  15938,  54181,  10454,  63706,  11912,  11085,  10123,  10627,  10240,    102,    125,  96946,  10120,  11170,  20493,  82951,    132,    122,  11057,  15003,  10120,  12764,  10216, 103730,  25130,  10340,    132,    122,  11057,  15003,  10120,  38356,  10115,    132,    124,  10854,  63641,  10116,  10120,  10304,  83106,  10567,    113, 107159,  33501,  17654,    114,    132,    122,  10854,  63641,  12349,  10120

In [ ]:
import re
error_count = 0
sample_count = 0
inference_progress_bar = tqdm(enumerate(recipe_list), total=len(recipe_list))
for idx, entry in inference_progress_bar:
    # print(idx, '++++++++++++++++++++++++++++++++++++++++++++++')
    entry_ingredient_strings_en = entry['text_en'].split(';')
    entry_ingredient_spans_en = []
    for ingredient_string_en in entry_ingredient_strings_en:
        start_index = entry['text_en'].find(ingredient_string_en)
        end_index = start_index + len(ingredient_string_en)
        entry_ingredient_spans_en.append([start_index, end_index])
    # print(entry_ingredient_spans_en)
    entry_ingredient_strings_it = entry['text_it'].split(';')
    entry_ingredient_spans_it = []
    for ingredient_string_it in entry_ingredient_strings_it:
        start_index = entry['text_it'].find(ingredient_string_it)
        end_index = start_index + len(ingredient_string_it)
        entry_ingredient_spans_it.append([start_index, end_index])
    # print(entry_ingredient_spans_it)

    for i, entity in enumerate(entry['entities_en']):
        sample_count += 1
        span_number = -1
        prediction_shift = 0

        # find in which ingredient span the entity is
        for j, span in enumerate(entry_ingredient_spans_en):
            if entity[0] >= span[0] and entity[1] <= span[1]:
                span_number = j
                break
        
        # define a shift to account for the length of the previous ingredient spans
        for j, span in enumerate(entry_ingredient_spans_en[:span_number]):
            prediction_shift += len(entry_ingredient_strings_it[j]) + 1

        search_span = entry_ingredient_spans_it[span_number]
        search_span_context = entry['text_it'][search_span[0]:search_span[1]]
        input = tokenizer(entry['text_en'][entity[0]:entity[1]], # query
                          search_span_context, # context
                          return_tensors='pt',
                          ).to(device)
        input_ids = input['input_ids'].squeeze()
        with torch.no_grad():
            outputs = model(**input)
        start_scores = outputs.start_logits
        end_scores = outputs.end_logits
        start_index_token = int(torch.argmax(start_scores))

        query_tokens = tokenizer(entry['text_en'][entity[0]:entity[1]], return_tensors='pt')['input_ids'].squeeze()
        
        if start_index_token < len(query_tokens):
            # print(f'################# SKIPPING: START INDEX IS WITHIN QUERY #################\nerrors: {error_count+1}\nerror rate = {error_count/sample_count}')
            error_count += 1
            continue

        if re.match(r'mbert.*|bert-base-multilingual-cased.*', model_name.split('/')[-1]):
            model_token_shift = 0
        elif re.match(r'mdeberta.*', model_name.split('/')[-1]):
            model_token_shift = 0

        end_index_token = int(torch.argmax(end_scores)) + model_token_shift
        decoded_input = ' '.join([f"({index}, {tokenizer.decode([id])})" for index, id in enumerate(input_ids)])
        token_based_prediction = tokenizer.decode(input['input_ids'].squeeze()[start_index_token:end_index_token])
        token_based_prediction_splitjoined = ''.join(token_based_prediction.split()).replace('#', '')

        if end_index_token < start_index_token:
            # print(f'################# SKIPPING: END INDEX TOKEN IS BEFORE START INDEX TOKEN #################\nerrors: {error_count+1}\nerror rate = {error_count/sample_count}')
            error_count += 1
            continue
        
        if start_index_token >= len(input_ids) - 1 or end_index_token > len(input_ids) - 1: # i put this here because the model sometimes predicts a token that is out of bounds
            # print(f'################# SKIPPING: OUT-OF-BOUNDS PREDICTION #################\nerrors: {error_count+1}\nerror rate = {error_count/sample_count}')
            error_count += 1
            continue

        char_span_start = input.token_to_chars(start_index_token)
        # the deberta tokenizer returns a space as well for words that are not at the start of the sentence, so here i am accounting for that, although not in an elegant way
        # later on i will look into how to instantiate the tokenizer so that it does not return the space, although the devs say that that reduces performance
        # so maybe that won't be a good idea
        if search_span_context[char_span_start[0]:char_span_start[0]+1] == ' ':
            char_span_start_adjusted = char_span_start[0] + 1 + prediction_shift
        else:
            char_span_start_adjusted = char_span_start[0] + prediction_shift
        char_span_end = input.token_to_chars(end_index_token - 1)

        if char_span_start is None or char_span_end is None:
            # print(f'################# SKIPPING: CHAR SPAN IS NONE #################\nerrors: {error_count+1}\nerror rate = {error_count/sample_count}')
            error_count += 1
            continue

        char_span_end_adjusted = char_span_end[1] + prediction_shift
        char_span_prediction = entry['text_it'][char_span_start_adjusted:char_span_end_adjusted]
        char_span_prediction_splitjoined = ''.join(char_span_prediction.split()).replace('#', '')
        char_span = (char_span_start_adjusted, char_span_end_adjusted)
        
        if not char_span[0] <= char_span[1]:
            # print(f'################# SKIPPING: CHAR SPAN END IS BEFORE CHAR SPAN START #################\nerrors: {error_count+1}\nerror rate = {error_count/sample_count}')
            error_count += 1
            continue

        error = False

        if char_span_prediction_splitjoined != token_based_prediction_splitjoined:
            print('ERROR: CHAR SPAN PREDICTION DOES NOT MATCH TOKEN-BASED PREDICTION')
            # print(f'################# SKIPPING #################\nerrors: {error_count+1}\nerror rate = {error_count/sample_count}')
            error = True
            error_count += 1
            print(f'recipe no.: {idx}, entity no.: {i}')
            print(entry['text_en'][entity[0]:entity[1]])
            print(entry['text_it'])
            print('span_number:', span_number)
            print('search_span:', search_span)
            print('prediction_shift:', prediction_shift)
            print('search_span_context:', search_span_context)
            print('query_tokens:', query_tokens)
            print('len(query_tokens):', len(query_tokens))
            print('start_index_token:', start_index_token)
            print('model_token_shift:', model_token_shift)
            print('end_index_token:', end_index_token)
            print('len(input_ids):', len(input_ids))
            print('encoding:', input_ids)
            print('decoded:', decoded_input)
            print('prediction_tokens:', input['input_ids'].squeeze()[start_index_token:end_index_token])
            print('token_based_prediction:', [token_based_prediction])
            print('token_based_prediction_splitjoined:', [token_based_prediction_splitjoined])
            print('gold:', [entry['text_en'][entity[0]:entity[1]]])
            print('char_span_start', char_span_start)
            print('char_span_start_adjusted', char_span_start_adjusted)
            print('char_span_end', char_span_end)
            print('char_span_end_adjusted:', char_span_end_adjusted)
            print('char_span_prediction:', [char_span_prediction])
            print('char_span_prediction_splitjoined:', [char_span_prediction_splitjoined])
            print('full context_en:', entry['text_en'])
            print('full context_it:', entry['text_it'])
            break
        
        entry['entities_it'].append([char_span[0], char_span[1], entry['entities_en'][i][2]])
        inference_progress_bar.set_postfix({'errors': error_count, 'error rate': round(error_count/sample_count, 3)})

        # prints go here
        # print('----------------------------------------------')
        # print(f'recipe no.: {idx}, entity no.: {i}')
        # print("search_span_context[char_span_start[0]:char_span_end[1]]", [search_span_context[char_span_start[0]:char_span_end[1]]])
        # print(entry['text_en'][entity[0]:entity[1]])
        # print(entry['text_it'])
        # print('span_number:', span_number)
        # print('search_span:', search_span)
        # print('prediction_shift:', prediction_shift)
        # print('search_span_context:', [search_span_context])
        # print('query_tokens:', query_tokens)
        # print('len(query_tokens):', len(query_tokens))
        # print('start_index_token:', start_index_token)
        # print('model_token_shift:', model_token_shift)
        # print('end_index_token:', end_index_token)
        # print('len(input_ids):', len(input_ids))
        # print('char_span_start', char_span_start)
        # print('char_span_start_adjusted', char_span_start_adjusted)
        # print('char_span_end', char_span_end)
        # print('char_span_end_adjusted:', char_span_end_adjusted)
        # print('full context:', entry['text_it'])
        # print('encoding:', input_ids)
        # print('decoded:', decoded_input)
        # print('prediction_tokens:', input['input_ids'].squeeze()[start_index_token:end_index_token])
        # print('char_span_prediction:', [char_span_prediction])
        # print('char_span_prediction_splitjoined:', [char_span_prediction_splitjoined]) 
        # print('token_based_prediction:', [token_based_prediction])
        # print('token_based_prediction_splitjoined:', [token_based_prediction_splitjoined])
        # print('gold:', [entry['text_en'][entity[0]:entity[1]]])
        
    if error:
        break 

print('error_count:', error_count)
print('sample_count:', sample_count)

In [ ]:
print(recipe_list[0])

In [ ]:
# save aligned dataset to a new json
import os
json_dir = '/home/pgajo/working/food/data/TASTEset/data'
json_filename_unformatted = f"{bilingual_path.split('/')[-1].replace('.json', '')}_{model_name.split('/')[-1]}_{(error_count/sample_count):.4f}.json".replace('unaligned', 'aligned')
new_data = {'classes': data['classes'], 'annotations': recipe_list}
with open(os.path.join(json_dir, json_filename_unformatted), 'w', encoding='utf8') as f:
    json.dump(new_data, f, ensure_ascii=False)
json_filename_unformatted

In [ ]:
# Save the dataset to a format which can be used in Label Studio
tasks = []
for recipe in recipe_list:
    predictions = []
    results = []
    languages = ['en', 'it']
    entry = {}
    for language in languages:
        for entity in recipe[f'entities_{language}']:
            results.append({
                'from_name': f'label_{language}',
                'to_name': f'text_{language}_ref',
                'type': 'labels',
                'value': {
                    'start': entity[0],
                    'end': entity[1],
                    'labels': [entity[2]]}
                    })
        entry[f'text_{language}'] = recipe[f'text_{language}']
    predictions.append({'model_version': model_name, 'result': results})
    tasks.append({
        'data': entry,
        'predictions': predictions,
    })

# Save Label Studio tasks.json


import os
json_dir = '/home/pgajo/working/food/data/TASTEset/data'
json_filename_ls = f"{bilingual_path.split('/')[-1].replace('.json', '')}_{model_name.split('/')[-1]}_{(error_count/sample_count):.4f}_{'-'.join(languages)}_labelstudio.json"
with open(os.path.join(json_dir, json_filename_ls), 'w', encoding='utf8') as f:
    json.dump(tasks, f, indent=4)
print(f'Save {len(tasks)} tasks to "{json_filename_ls}"')